In [3]:
import os
import shutil
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet152
from tensorflow.keras import layers, models
from sklearn.model_selection import ParameterGrid
import numpy as np

# Function to build the model
def build_model(num_classes, custom_layers):
    tf.keras.backend.clear_session()

    # Load ResNet152 as the base model
    base_model = ResNet152(weights='imagenet', include_top=False, input_shape=input_shape)

    # Freeze the base model
    base_model.trainable = False

    # Create the model
    model = models.Sequential()
    model.add(base_model)
    model.add(layers.GlobalAveragePooling2D())
    
    # Add custom layers
    for layer in custom_layers:
        model.add(layer)

    model.add(layers.Dense(num_classes, activation='softmax'))

    # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return model

# Assuming dataset_folder is defined and points to your dataset directory
dataset_folder = 'C:\\Users\\Alexandra\\Documents\\SPRING_2024\\BMEN_689\\Project_2\\Jupyter_Notebook\\With_Pre_Processing\\New512by512\\'

# Function to split dataset into train/validation/test
def split_dataset(data_folder, train_size=0.6, val_size=0.2, test_size=0.2):
    for class_folder in os.listdir(data_folder):
        class_path = os.path.join(data_folder, class_folder)
        if os.path.isdir(class_path):
            files = os.listdir(class_path)
            train_files, test_files = train_test_split(files, test_size=val_size+test_size, random_state=42)
            val_files, test_files = train_test_split(test_files, test_size=test_size/(val_size+test_size), random_state=42)

            train_folder = os.path.join(data_folder, 'train', class_folder)
            val_folder = os.path.join(data_folder, 'validation', class_folder)
            test_folder = os.path.join(data_folder, 'test', class_folder)
            for folder in [train_folder, val_folder, test_folder]:
                if not os.path.exists(folder):
                    os.makedirs(folder)

            for file in train_files:
                shutil.copy(os.path.join(class_path, file), os.path.join(train_folder, file))
            for file in val_files:
                shutil.copy(os.path.join(class_path, file), os.path.join(val_folder, file))
            for file in test_files:
                shutil.copy(os.path.join(class_path, file), os.path.join(test_folder, file))

# Call the function to split the dataset
split_dataset(dataset_folder)

input_shape = (512, 512, 3)  # Use 3 channels for RGB images
num_classes = 3

# Define directory paths for data generators
train_dir = os.path.join(dataset_folder, 'train')
validation_dir = os.path.join(dataset_folder, 'validation')

# Setup data generators
train_datagen = ImageDataGenerator()
validation_datagen = ImageDataGenerator()

batch_size = 16
target_size = (512, 512)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=target_size,
    batch_size=batch_size,
    color_mode='rgb',
    class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=target_size,
    batch_size=batch_size,
    color_mode='rgb',
    class_mode='categorical')

# Define a parameter grid for custom layers
param_grid = {
    'custom_layers': [
        [layers.Dense(512, activation='relu'), layers.Dropout(0.2)],
        [layers.Dense(512, activation='relu'), layers.Dropout(0.4)],
        [layers.Dense(512, activation='relu'), layers.Dropout(0.6)],
        [layers.Dense(256, activation='relu'), layers.Dropout(0.2)],
        [layers.Dense(256, activation='relu'), layers.Dropout(0.4)],
        [layers.Dense(256, activation='relu'), layers.Dropout(0.6)],
        [layers.Dense(128, activation='relu'), layers.Dropout(0.2)],
        [layers.Dense(128, activation='relu'), layers.Dropout(0.4)],
        [layers.Dense(128, activation='relu'), layers.Dropout(0.6)],
        [layers.Dense(512, activation='relu')],
        [layers.Dense(256, activation='relu')],
        [layers.Dense(128, activation='relu')],
        [layers.Dropout(0.2)],
        [layers.Dropout(0.4)],
        [layers.Dropout(0.6)],
        []  # No custom layers
    ]
}

# Generate all possible combinations of parameters
grid = ParameterGrid(param_grid)

best_score = -np.inf
best_params = None

for params in grid:
    # Build the model with current parameters
    model = build_model(num_classes, params['custom_layers'])

    # Fit the model
    history = model.fit(
        train_generator,
        steps_per_epoch=len(train_generator),
        validation_data=validation_generator,
        validation_steps=len(validation_generator),
        epochs=10,
        verbose=1
    )

    # Evaluate the model
    score = model.evaluate(validation_generator, verbose=0)[1]

    # Check if this configuration is the best so far
    if score > best_score:
        best_score = score
        best_params = params

# Print the best parameters
print("Best parameters:", best_params)


Found 180 images belonging to 3 classes.
Found 60 images belonging to 3 classes.


Epoch 1/10


12/12 [==============================] - 39s 3s/step - loss: 1.0517 - accuracy: 0.6111 - val_loss: 0.4905 - val_accuracy: 0.7667
Epoch 2/10
12/12 [==============================] - 34s 3s/step - loss: 0.2951 - accuracy: 0.8833 - val_loss: 0.1355 - val_accuracy: 0.9833
Epoch 3/10
12/12 [==============================] - 34s 3s/step - loss: 0.1420 - accuracy: 0.9611 - val_loss: 0.0950 - val_accuracy: 0.9667
Epoch 4/10
12/12 [==============================] - 34s 3s/step - loss: 0.0904 - accuracy: 0.9833 - val_loss: 0.0602 - val_accuracy: 0.9833
Epoch 5/10
12/12 [==============================] - 34s 3s/step - loss: 0.0602 - accuracy: 0.9889 - val_loss: 0.0520 - val_accuracy: 0.9833
Epoch 6/10
12/12 [==============================] - 34s 3s/step - loss: 0.0536 - accuracy: 0.9889 - val_loss: 0.0644 - val_accuracy: 0.9833
Epoch 7/10
12/12 [==============================] - 34s 3s/step - loss: 0.0